# 动态规划
- 最重要是写出状态转移方程
- 首先思考怎么暴力穷举
- 然后两种优化思路：
    - 带备忘录的递归解法（自顶向下）
    - dp数组的迭代解法（自底向上）

- 找到变化的状态
- 做选择

## 题型
- 最长递增子序列长， 最大子数组和 使用一维dp
- 最长公共子序列，最短编辑距离 使用二维dp
- **特别注意***用索引取序列值的时候，是否要-1，因为索引第一位对应空字符，输出的时候也要注意索引取值取不到len(s)

## 子序列+最值  = 动态规划
- 因为子序列不要求连续，穷举都是指数级复杂度，基本上大概率要用dp优化
### 两种模板：
- 一个一维dp数组：（dp[i]定义为以nums[i]结尾的目标序列相关值）
    - 最长递增子序列
    - 最大子数组和

- 一个二维dp数组：
    - 涉及两个字符串/数组：（dp[i][j]定义为s1[:i], s2[:j]的相关目标值）
        - 最长公共子序列
        - 最短编辑距离
    - 涉及一个字符串/数组：（dp[i][j]定义为s[i:j]相关目标值）
        - 最长回文子序列
        - 构造回文串的最小插入次数


# 凑零钱
- 给可用的零钱面值列表，目标金额数，问最小多少个零钱能凑出

In [ ]:
# 思路：目标金额不管有多少种凑法，凑的最后一枚肯定是从零钱列表里面选一个出来的（做选择）。
# 那么在做选择的时候，子问题就变成了要凑出"amount-选择" 这个目标金额的问题

class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        dp = [amount+1 for i in range(amount + 1)]  # 目标金额为i时，最小零钱数为dp[i]，初始化为amount+1等价于无穷大，因为最多只需要amount枚一元硬币能凑出来
        dp[0] = 0
        for i in range(len(dp)):
            for coin in coins:
                if i < coin: # 目标金额数比coin还小
                    continue
                dp[i] = min(dp[i], 1+dp[i-coin])
        return dp[amount] if dp[amount]!= amount+1 else -1

## 2.1 最长递增子序列
- 给一个无序的数组，返回其最长递增子序列的长度

In [ ]:
# 思路：dp[i]表示以nums[i]结尾的最长递增子序列长度
# 当已知前面的结果，当前以nums[i]结尾的子序列有很多，因为是递增，所以nums[i]只会接在比他小的数后面，使得子序列长度+1；
# 此时就要穷举，在这些以nums[i]结尾的子序列中最大长度是多少，作为dp[i]的值；完成状态转移

def solution(nums):
    if not nums:
        return 0
    dp = [1 for i in range(len(nums))]
    for i in range(1, len(nums)): # 逐个计算dp中的值
        # 穷举以nums[i]结尾的递增子序列，只有比nums[i]值小的前面值才有机会
        for j in range(i):
            if nums[i] > nums[j]:
                dp[i] = max(dp[i], dp[j]+1)
    return max(dp)    ## 输出的是其中最大的       

## 2.3 最大子数组和
- 给一个整数数组，找出一个和最大的子数组，返回这个子数组的和
- 子数组类似子串，要连续

In [ ]:
# 思路同上，dp[i]表示以nums[i]结尾的最大子数组和
# 已知dp[i-1]，到i位置，dp[i]有两种选择，一个是带上nums[i]新的子数组和更大，一种是不带前面的子数组自成一派，nums[i]个人就最大
# 选哪个？当然是更大的那种情况
def solution(nums):
    dp = [None]*len(nums)
    dp[0] = nums[0]
    for i in range(1,len(nums)):
        dp[i] = max(dp[i-1]+nums[i], nums[i])
    return max(dp)


## 2.5 最长公共子序列
- 给俩个字符串，求最长的公共子序列长度
- 子序列不需要连续

In [ ]:
# 思路：二维dp[i][j]表示s1[0...i] 和s2[0...j]中最长公共子序列的长度
# 那么考虑s1[i]和s2[j] 的关系，如果二者相等，那么一定都在新的公共子序列中，就可用dp[i-1][j-1]+1 得到dp[i][j]
# 如果二者不相等，那么至少有一个是不在新的公共子序列中的，那么我们新的最长公共子序列长度就取更大的

def solution(s1, s2):
    m = len(s1)
    n = len(s2)
    dp = [[0 for j in range(n+1)] for i in range(m+1)] # 行和列多加一个空串的情况为basecase
    for i in range(m+1):
        dp[i][0] = 0
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:  #TODO 注意这里是i=0和j=0对应的都是空字符，所以实际根据索引取字符的时候要i-1，j-1
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return dp[m][n]    

## 2.6最小编辑距离

In [ ]:
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        m, n = len(word1), len(word2)
        dp = [[0 for j in range(n+1)] for i in range(m+1)]
        for i in range(1,m+1):
            dp[i][0] = i
        for j in range(1,n+1):
            dp[0][j] = j

        for i in range(1, m+1):
            for j in range(1, n+1):
                if word1[i-1] == word2[j-1]:  # [ ]这里取值最容易错
                    dp[i][j] = dp[i-1][j-1] 
                else:
                    dp[i][j] = min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1]) +1
        return dp[m][n]


## 最长回文子序列

In [ ]:
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        dp = [[0 for j in range(len(s))] for i in range(len(s))]
        for i in range(len(s)):
            for j in range(len(s)):
                if i==j:
                    dp[i][j] = 1
        for i in range(len(s)-2, -1, -1):  # 由于每个位置是由左下的值推出来的，所以遍历方向：从下往上，从左往右
            for j in range(i+1, len(s)):
                if s[i] == s[j]:
                    dp[i][j] = dp[i+1][j-1] + 2
                else:
                    dp[i][j] = max(dp[i+1][j], dp[i][j-1])
        return dp[0][len(s)-1]

## 构造回文串的最小插入次数

In [ ]:
# 思路，由s[i+1:j-1] 推s[i:j]

def solution(s):
    n = len(s)
    dp = [[0 for j in range(n)] for i in range(n)]
    for i in range(n-2, -1, -1):  # 由于每个位置是由左下的值推出来的，所以遍历方向：从下往上，从左往右
        for j in range(i+1, n):
            if s[i] == s[j]:
                dp[i][j] = dp[i+1][j-1]
            else:
                dp[i][j] = min(dp[i+1][j], dp[i][j-1]) + 1
    return dp[0][n-1]

## 背包类型问题

- 0-1背包问题

- 明确状态和选择
    - 状态：在这个问题中，背包的总容量可以变化，可选择的物品可以变化
            因此，状态有两个：背包的容量，可选择的物品
           目标最大价值不是状态，他的变化是目标值的变化 
    - 选择：对于每件物品，只有两个选择，装入背包或不装入

- 两个状态，需要一个二维的db table
  for 状态1 in 状态1的所有取值：
    for 状态2 in 状态2的所有取值：
        for ...
            dp[状态1][状态2][...] = 择优(选择1，选择2...)



In [ ]:
def knapsack(W: int, N: int, wt: List[int], val: List[int]) -> int:
    assert N == len(wt)
    # 初始化一个二维数组，用于存储状态
    # dp[i][j] 表示将前 i 个物品装入容量为 j 的背包中所获得的最大价值
    dp = [[0] * (W + 1) for _ in range(N + 1)]
    # 开始进行递推
    for i in range(1, N + 1):
        for w in range(1, W + 1):
            if w - wt[i - 1] < 0: #第i物品再wt中索引是i-1 #[ ]
                # 当前商品 i 的重量已经超过了 w，无法被放入当前容量为 w 的背包中，只能选择不装入背包
                dp[i][w] = dp[i - 1][w]
            else:
                # 当前商品 i 的重量小于等于当前容量 w，可以尝试将其放入背包中
                # 取最大值，考虑是将其放入之前的最优方案中还是选择不放
                dp[i][w] = max(
                    dp[i - 1][w - wt[i - 1]] + val[i - 1], #放入
                    dp[i - 1][w]   #没放入
                )
    # 返回最大价值
    return dp[N][W]


## 子集背包问题
- 

In [ ]:

def canPartition(nums: List[int]) -> bool:
    sum = 0
    for num in nums:  # 遍历计算数组总和
        sum += num
    # 和为奇数时，不可能划分成两个和相等的集合
    if sum % 2 != 0:
        return False
    n = len(nums)
    sum = sum // 2
    # 定义二维数组 dp
    dp = [[False] * (sum + 1) for _ in range(n + 1)]
    # base case
    for i in range(n + 1):
        dp[i][0] = True

    for i in range(1, n + 1):
        for j in range(1, sum + 1):
            if j - nums[i - 1] < 0:
                # 背包容量不足，不能装入第 i 个物品
                dp[i][j] = dp[i - 1][j]
            else:
                # 装入或不装入背包
                dp[i][j] = dp[i - 1][j] or dp[i - 1][j - nums[i - 1]]
    return dp[n][sum]


## 完全背包问题
    - dp[i][j]表示给你前i中硬币类型，恰好装满j重量的可能装法
    - 如果没有使用第i类硬币，dp[i][j] = dp[i-1][j]
        - 意思是你提供了总共i种硬币，如果我选择不使用第i种硬币，就相当于只提供了i-1种硬币要凑出j重量的情况
    - 如果使用了第i类硬币，dp[i][j] = dp[i][j-coin[i-1]]
        - 意思是：由于每种硬币数量都是无限的，既然我用了第i种硬币，那么找一个更小规模的问题可以是：你提供了总共i种硬币，但是只要我凑出j-coin[i-1]数量，那么如果我知道了这种情况的结果，只要在每个结果基础上加一个coin[i-1]这枚硬币，必然凑出了j数量，子问题的解等价于当前问题的解
    - 最后题目要求总共的可能组合数量，对上述两种情况做求和


In [ ]:
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        n = len(coins)
        # dp[i][j]表示给你前i个coin，凑出j金额的方法数量
        dp = [[0]*(amount+1) for i in range(n+1)]
        for i in range(n+1):
            dp[i][0] = 1
        for i in range(1, n+1):
            for j in range(1, amount+1):
                if coins[i-1] <= j: #能凑出的可能
                    dp[i][j] = dp[i-1][j] + dp[i][j-coins[i-1]] #注意coin数量无限
                else:
                    dp[i][j] = dp[i-1][j]
        return dp[n][amount]